# 상가 점포명 모음 

In [2]:
!pwd

/Users/wonny/commercial_district_analysis/kimwon/NaverMap


In [19]:
import numpy as np

In [20]:
import pandas as pd

df = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_서울_202212.csv')

In [21]:
df.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')

In [22]:
df.isnull().sum()

상가업소번호            0
상호명               0
지점명          314555
상권업종대분류코드         0
상권업종대분류명          0
상권업종중분류코드         0
상권업종중분류명          0
상권업종소분류코드         0
상권업종소분류명          0
표준산업분류코드      26181
표준산업분류명       26181
시도코드              0
시도명               0
시군구코드             0
시군구명              0
행정동코드             0
행정동명              0
법정동코드             0
법정동명              0
지번코드              0
대지구분코드            0
대지구분명             0
지번본번지             0
지번부번지         78602
지번주소              0
도로명코드             0
도로명               0
건물본번지             0
건물부번지        327127
건물관리번호            0
건물명          190057
도로명주소             0
구우편번호             0
신우편번호            36
동정보          333667
층정보          146271
호정보          377724
경도                0
위도                0
dtype: int64

In [23]:
df = df.loc[df['상권업종대분류명'] == '음식']  
df= df[['상호명','행정동명','행정동코드','법정동코드']]
df

,상호명,행정동명,행정동코드,법정동코드
4,비지트,방배4동,1165062100,1165010100
5,쓰리에프,사당2동,1159063000,1159010700
6,다향,서초3동,1165053000,1165010800
7,고향생막걸리,압구정동,1168054500,1168010700
12,도전최강달인왕만두,고덕2동,1174056000,1174010200
...,...,...,...,...
377706,텐퍼센트커피,시흥1동,1154567000,1154510300
377710,청담우애,청담동,1168056500,1168010400
377712,막리단길,동선동,1129057500,1129011600
377716,보름설기,신도림동,1153051000,1153010100


In [24]:
df.to_csv('상호명_동_행정동_법정동코드.csv',index=False)

In [16]:
df= df.iloc[:100]
df

,상호명,행정동명,행정동코드,법정동코드
4,비지트,방배4동,1165062100,1165010100
5,쓰리에프,사당2동,1159063000,1159010700
6,다향,서초3동,1165053000,1165010800
7,고향생막걸리,압구정동,1168054500,1168010700
12,도전최강달인왕만두,고덕2동,1174056000,1174010200
...,...,...,...,...
431,김밥천국등촌,등촌3동,1150053500,1150010200
432,가정식백반,양재1동,1165065100,1165010200
433,온파스타화곡점,화곡8동,1150059300,1150010300
434,기사식당,제기동,1123054500,1123010300


# 크롤링

In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time


chromedriver = 'kimwon/chromedriver' 
driver = webdriver.Chrome(chromedriver) 


# 네이버 지도 검색창에 [~동 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.

df['naver_keyword'] = df['행정동명'] + "%20" + df['상호명']  # "%20"는 띄어쓰기를 의미합니다.
df['naver_map_url'] = ''


# 본격적으로 가게 상세페이지의 URL을 가져옵시다

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(1)
        df.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR,"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR, "#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()


# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']


# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naver_map_url'].isnull()]

/var/folders/s3/xdg3f00d5gz2dnfsw7wsjyjc0000gn/T/ipykernel_75337/1229034579.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)
/var/folders/s3/xdg3f00d5gz2dnfsw7wsjyjc0000gn/T/ipykernel_75337/1229034579.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_keyword'] = df['행정동명'] + "%20" + df['상호명']  # "%20"는 띄어쓰기를 의미합니다.
/var/folders/s3/xdg3f00d5gz2dnfsw7wsjyjc0000gn/T/ipykernel_75337/1229034579.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

이번에 찾을 키워드 : 0 / 99 행 방배4동%20비지트
이번에 찾을 키워드 : 1 / 99 행 사당2동%20쓰리에프
Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=110.0.5481.96)
Stacktrace:
0   chromedriver                        0x0000000100fd1138 chromedriver + 4923704
1   chromedriver                        0x0000000100f499d3 chromedriver + 4368851
2   chromedriver                        0x0000000100b92787 chromedriver + 472967
3   chromedriver                        0x0000000100bd6ba3 chromedriver + 752547
4   chromedriver                        0x0000000100bd6df1 chromedriver + 753137
5   chromedriver                        0x0000000100c1d734 chromedriver + 1042228
6   chromedriver                        0x0000000100bfffdd chromedriver + 921565
7   chromedriver                        0x0000000100c1ac0f chromedriver + 1031183
8   chromedriver

/var/folders/s3/xdg3f00d5gz2dnfsw7wsjyjc0000gn/T/ipykernel_75337/1229034579.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']


In [18]:
df

,상호명,행정동명,행정동코드,법정동코드,naver_keyword,naver_map_url
4,비지트,방배4동,1165062100,1165010100,방배4동%20비지트,https://m.place.naver.com/restaurant/1848424123
7,고향생막걸리,압구정동,1168054500,1168010700,압구정동%20고향생막걸리,https://m.place.naver.com/restaurant/1496470647
12,도전최강달인왕만두,고덕2동,1174056000,1174010200,고덕2동%20도전최강달인왕만두,https://m.place.naver.com/restaurant/1187890786
13,L.A.D,서교동,1144066000,1144012000,서교동%20L.A.D,https://m.place.naver.com/restaurant/1857845585
14,양푼이김치찌개,방이2동,1171056200,1171011100,방이2동%20양푼이김치찌개,https://m.place.naver.com/restaurant/1403941863
...,...,...,...,...,...,...
429,플라토어학학원,홍은2동,1141068500,1141011800,홍은2동%20플라토어학학원,https://m.place.naver.com/restaurant/31850389
431,김밥천국등촌,등촌3동,1150053500,1150010200,등촌3동%20김밥천국등촌,https://m.place.naver.com/restaurant/1784619708
432,가정식백반,양재1동,1165065100,1165010200,양재1동%20가정식백반,https://m.place.naver.com/restaurant/1819930583
434,기사식당,제기동,1123054500,1123010300,제기동%20기사식당,https://m.place.naver.com/restaurant/36947275


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합칠 것입니다.

naver_map_name_list = []
naver_map_type_list = []
blog_review_list = []
blog_review_qty_list = []
naver_map_star_review_stars_list = []
naver_map_star_review_qty_list = []
chromedriver = 'chromedriver' 

# 메인 드라이버 : 별점 등을 크롤링
driver = webdriver.Chrome(chromedriver) 

# 서브 드라이버 : 블로그 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
sub_driver = webdriver.Chrome(chromedriver)

for i, url in enumerate(tqdm(df['naver_map_url'])):

    driver.get(url)
    sub_driver.get(url+"/review/ugc")
    time.sleep(1)


    try:

        # 간단 정보 가져오기
        
        # 네이버 지도의 유형 분류
        naver_map_type = driver.find_element(By.CSS_SELECTOR,"#_title > span._3ocDE").text

        # 블로그 리뷰 수
        blog_review_qty = driver.find_element(By.CSS_SELECTOR,"#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(3) > a > em").text

        # 블로그 별점 점수
        star_review_stars = driver.find_element(By.CSS_SELECTOR,"#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span._1Y6hi._1A8_M > em").text

        # 블로그 별점 평가 수
        star_review_qty = driver.find_element(By.CSS_SELECTOR,"#app-root > div > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(2) > a > em").text
       

        # 블로그 리뷰 텍스트 가져오기
        review_text_list = [] # 임시 선언

        
        # 네이버 지도 블로그 리뷰 탭은 동적 웹사이트의 순서가 주문하기, 메뉴보기 등의 존재 여부로 다르기 때문에 css selector가 아니라 element 찾기로 진행
        review_text_crawl_list = sub_driver.find_element(By.CLASS_NAME,"_2CbII")
        
        # find element's' 메소드를 통해 가져온 내용은 리스트로 저장되고, 리스트 타입을 풀어서(for문 사용) 임시 데이터에 모아 두어야 한다
        for review_crawl_data in review_text_crawl_list:
            review_text_list.append(review_crawl_data.find_element(By.TAG_NAME,'div').text)
        
        # 그 리스트에 저장된 텍스트 (한 식당에 대한 여러 리뷰들)를 한 텍스트 덩어리로 모아(join)줍니다.
        review_text = ','.join(review_text_list)


        blog_review_list.append(review_text)

        naver_map_type_list.append(naver_map_type)
        blog_review_qty_list.append(blog_review_qty)
        naver_map_star_review_stars_list.append(star_review_stars)
        naver_map_star_review_qty_list.append(star_review_qty)

    # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
    except Exception as e1:
        print(f"{i}행 문제가 발생")
        
        # 리뷰가 없으므로 null을 임시로 넣어줍니다.
        blog_review_list.append('null')  
        naver_map_type_list.append('null')
        blog_review_qty_list.append('null')
        naver_map_star_review_stars_list.append('null')
        naver_map_star_review_qty_list.append('null')


        
driver.quit()
sub_driver.quit()


df['naver_store_type'] = naver_map_type_list  # 네이버 상세페이지에서 크롤링한 업체 유형
df['naver_star_point'] = naver_map_star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점
df['naver_star_point_qty'] = naver_map_star_review_qty_list  # 네이버 상세페이지에서 별점 평가를 한 횟수
df['naver_blog_review_txt'] = blog_review_list  # 네이버 상세페이지에 나온 블로그 리뷰 텍스트들
df['naver_blog_review_qty'] = blog_review_qty_list  # 네이버 상세페이지에 나온 블로그 리뷰의 총 개수

In [ ]:

# 크롤링 에러가 떠서 'null'을 넣어 둔 데이터는 활용 의미가 없으므로 행 삭제를 해줘도 됩니다
df = df.loc[~(df['naver_store_type'].str.contains('null'))]


# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 바꿔줍니다.
df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']] = df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']].apply(pd.to_numeric)
